In [1]:
# Чтение вслух
import os
import re
from pygame import mixer
import datetime
import time
from gtts import gTTS
import json

# Для того чтобы не возникало коллизий при удалении mp3 файлов
# заведем переменную mp3_nameold в которой будем хранить имя предыдущего mp3 файла
mp3_nameold='111'
mp3_name = "1.mp3"

# Инициализируем звуковое устройство
mixer.init()

# Открываем файл с текстом и по очереди читаем с него строки в ss
f = open("from.txt","r")
ss = f.readline()
while ss:
    # Делим прочитанные строки на отдельные предложения
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(ss)])

    # Перебираем массив с предложениями 
    for x in sentences:
        if(x!=""):
            print(x)
            # Эта строка отправляет предложение которое нужно озвучить гуглу
            tts=gTTS(text=x, lang='ru', slow=False)
            # Получаем от гугла озвученное предложение в виде mp3 файла           
            tts.save(mp3_name)
            # Проигрываем полученный mp3 файл
            mixer.music.load(mp3_name)
            mixer.music.play()
            while mixer.music.get_busy():
                time.sleep(0.1)
            # Если предыдущий mp3 файл существует удаляем его
            # чтобы не захламлять папку с приложением кучей mp3 файлов
            if(os.path.exists(mp3_nameold) and (mp3_nameold!="1.mp3")):
                os.remove(mp3_nameold)
            mp3_nameold=mp3_name
            # Формируем имя mp3 файла куда будет сохраняться озвученный текст текущего предложения
            # В качестве имени файла используем текущие дату и время
            now_time = datetime.datetime.now()
            mp3_name = now_time.strftime("%d%m%Y%I%M%S")+".mp3"
            
    # Читаем следующую порцию текста из файла
    ss = f.readline()

# Закрываем файл    
f.close

# Устанвливаем текущим файлом 1.mp3 и закрываем звуковое устройство
# Это нужно чтобы мы могли удалить предыдущий mp3 файл без колизий
mixer.music.load('1.mp3')
mixer.stop
mixer.quit

# Удаляем последний предыдущий mp3 файл
if(os.path.exists(mp3_nameold)):
    os.remove(mp3_nameold)


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
Здраствуйте, Вы дозвонились в такси "Сын маминой подруги", Где вы находитесь


/home/hetag/anaconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


In [2]:
import pygame
import time
pygame.init()
pygame.mixer.music.load("2.wav")
pygame.mixer.music.play()
time.sleep(10)

import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile("2.wav") as source:
    audio = r.record(source)

try:
    s = r.recognize_google(audio, language="ru-RU")
    print("Text: "+s)
except Exception as e:
    print("Exception: "+str(e))

z = open('text-from.txt','w') 
z.write(s) 
z.close()

Text: я нахожусь на улице шмулевича 5


In [3]:
from bs4 import BeautifulSoup
import requests

streetsList = []

def get_html(url):
    r = requests.get(url)
    return r.text

def get_total_pages( html ):
    soup = BeautifulSoup(html, 'lxml')
    pages = int(soup.find('ul', class_='pagination').find_all('a')[-1].text)
    print(pages)
    return pages

def get_street_names( html ):
    soup = BeautifulSoup(html, 'lxml')
    pageForFinding = soup.find('div', class_='content-block').find_all('a')
    for street in pageForFinding:
        streetsList.append(street.text)
   # pageForFinding = pageForFinding.strip()

def main():
    url = "https://mapdata.ru/severnaya-osetiya/vladikavkaz/ulicy/stranica-1/"
    html_text = get_html(url)
    pages = get_total_pages(html_text)
    for i in range (1, pages+1):
        urlForEachPage = 'https://mapdata.ru/severnaya-osetiya/vladikavkaz/ulicy/stranica-' + str(i) + '/'
        html_text = get_html(urlForEachPage)
        get_street_names(html_text)

    f = open('outputdata.txt', 'w')
    for item in streetsList:
        f.write("%s\n" % item)
    f.close()
    print(len(streetsList))

if __name__ == '__main__':
    main()

5
394


In [4]:
string1 = open('text-from.txt', 'r')
text = ((string1.read()).lower()).split(' ')
list1 = open('outputdata.txt', 'r')
text2 = ((list1.read()).lower()).split()

for i in range(len(text2)):
    if text2[i] == 'улица':
        text2[i] = ''

for x in range(len(text)):
    for p in text2:
        if text[x] == p:
            text[x] = 'улица  ' + text[x].title()

text = str(text)
text = text.replace('[','').replace(']','').replace(',','').replace(r"'","")
        
d = open('text-from.txt','w') 
d.write(text) 
d.close()

In [5]:
from natasha import (AddressExtractor, LocationExtractor)
from natasha.markup import show_markup, show_json, format_json


extractor = AddressExtractor()

t = open('text-from.txt', 'r')
text = t.read()

matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
json_string = format_json(facts)

я нахожусь на улице [[улица  Шмулевича 5]]


In [6]:
json_string = format_json(facts)
parsed_string = json.loads(json_string)
for i in parsed_string:
    for j in i:
        print('Клиент находится на улице:')
        for k in i[j]:
            for p in k:
                print(p, k[p])

Клиент находится на улице:
name Шмулевича
type улица
number 5


In [7]:
# Для того чтобы не возникало коллизий при удалении mp3 файлов
# заведем переменную mp3_nameold в которой будем хранить имя предыдущего mp3 файла
mp3_nameold='111'
mp3_name = "1.mp3"

# Инициализируем звуковое устройство
mixer.init()

# Открываем файл с текстом и по очереди читаем с него строки в ss
f = open("to.txt","r")
ss = f.readline()
while ss:
    # Делим прочитанные строки на отдельные предложения
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(ss)])

    # Перебираем массив с предложениями 
    for x in sentences:
        if(x!=""):
            print(x)
            # Эта строка отправляет предложение которое нужно озвучить гуглу
            tts=gTTS(text=x, lang='ru', slow=False)
            # Получаем от гугла озвученное предложение в виде mp3 файла           
            tts.save(mp3_name)
            # Проигрываем полученный mp3 файл
            mixer.music.load(mp3_name)
            mixer.music.play()
            while mixer.music.get_busy():
                time.sleep(0.1)
            # Если предыдущий mp3 файл существует удаляем его
            # чтобы не захламлять папку с приложением кучей mp3 файлов
            if(os.path.exists(mp3_nameold) and (mp3_nameold!="1.mp3")):
                os.remove(mp3_nameold)
            mp3_nameold=mp3_name
            # Формируем имя mp3 файла куда будет сохраняться озвученный текст текущего предложения
            # В качестве имени файла используем текущие дату и время
            now_time = datetime.datetime.now()
            mp3_name = now_time.strftime("%d%m%Y%I%M%S")+".mp3"
            
    # Читаем следующую порцию текста из файла
    ss = f.readline()

# Закрываем файл    
f.close

# Устанвливаем текущим файлом 1.mp3 и закрываем звуковое устройство
# Это нужно чтобы мы могли удалить предыдущий mp3 файл без колизий
mixer.music.load('1.mp3')
mixer.stop
mixer.quit

# Удаляем последний предыдущий mp3 файл
if(os.path.exists(mp3_nameold)):
    os.remove(mp3_nameold)


Куда отправитесь


/home/hetag/anaconda3/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


In [8]:
import pygame
import time
pygame.init()
pygame.mixer.music.load("3.wav")
pygame.mixer.music.play()
time.sleep(10)

import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile("3.wav") as source:
    audio = r.record(source)

try:
    s = r.recognize_google(audio, language="ru-RU")
    print("Text: "+s)
except Exception as e:
    print("Exception: "+str(e))

z = open('text-to.txt','w') 
z.write(s) 
z.close()

Text: Я поеду на Тельмана 3


In [9]:
string1 = open('text-to.txt', 'r')
text = ((string1.read()).lower()).split(' ')
list1 = open('outputdata.txt', 'r')
text2 = ((list1.read()).lower()).split()

for i in range(len(text2)):
    if text2[i] == 'улица':
        text2[i] = ''

for x in range(len(text)):
    for p in text2:
        if text[x] == p:
            text[x] = 'улица  ' + text[x].title()

text = str(text)
text = text.replace('[','').replace(']','').replace(',','').replace(r"'","")
        

d = open('text-to.txt','w') 
d.write(text) 
d.close()

In [10]:
from natasha import (AddressExtractor, LocationExtractor)
from natasha.markup import show_markup, show_json


extractor = AddressExtractor()

t = open('text-to.txt', 'r')
text = t.read()

matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
json_string = format_json(facts)

я поеду на [[улица  Тельмана 3]]


In [11]:
json_string = format_json(facts)
parsed_string = json.loads(json_string)
for i in parsed_string:
    for j in i:
        print('Клиент поедет на улицу:')
        for k in i[j]:
            for p in k:
                print(p, k[p])

Клиент поедет на улицу:
name Тельмана
type улица
number 3
